# Isotank Design for the LV4 Sub-orbital Rocket
## Motivation
With liquid oxygen (LOX) being one of the most common oxidizers used in rocketry, lightweight cryogenic storage are a critical design component. Traditionally metallic materials, such as stainless steel, aluminimun and aluminum-lithium alloys have been partially due to their compatibility with LOX. Composite materials have also been looked at over the years, but finding specific materials that can withstand both the cold temperatures and remain non-reactive can be difficult. As part of the PSAS LV4 program, a capstone team at PSU built a small-scale composite tank design, but there are concerns with scaling up the design to LV4 scales.

In light of the difficulties manufacturing previous designs, a new approach for LV4 LOX tanks using aluminum isogrids is being studied. The goal is to have a design that is manufacturable by the amatuer group, but has a mass close to composite designs. 